## Inverse Problem

This notebook is a simple example of how to solve an inverse problem using pyshdom. It assumes that we have run the 'SimulateRadiances' notebook and saved the result.

In [1]:
#imports

import pyshdom
import numpy as np
import xarray as xr
from collections import OrderedDict
import pylab as py
np.random.seed(1)

### Load Measurements

Here we load the synthetic measurements and also all of the inputs to the solver. We will use several of these inputs to perform 'inverse crimes' whereby we fix some aspects of the problem perfectly as in the forward simulation. When using real world measurements no such short cuts are necessary.

* First, we have to look at the measurements and select a region of interest and define our `rte_grid`. 
* Then we have to model the sensor sub-pixel geometry.
* We need to analyze whether the grid and sensor geometry are consistent. The SpaceCarver is useful for this. If not then we may need to change resolution of `rte_grid` to match the resolution of the measurements etc.
* We need to decide how to represent the surface, which is currently fixed.
* We need to decide how to represent the atmosphere.
    * What scattering species are we modelling? What are their optical models? 
    * Which quantities will be unknowns and which are fixed?
    * What will be the abstract state that we will reconstruct? For this we need to set the mapping between the abstract state and the RTE solver.
* Now that we are organized we need to initialize our state vector of unknowns. The method for this initialization may itself be quite involved as starting nearer the answer is better. This typically goes hand in hand with the selection of any fixed variables.
* Lastly we perform the optimization.

In this tutorial we will reconstruct only the `extinction` and we will use forward quantities from the ground truth synthetic measurements for simplicity.

In [2]:
sensors, solvers, rte_grid = pyshdom.util.load_forward_model('./SimulateRadiances.nc')

0.1 0.015601813796583135 False False


In [3]:
sensor_list = []
for sensor in sensors['MSPI']['sensor_list']:
    copied = sensor.copy(deep=True)
    weights = np.zeros(sensor.sizes['nrays'])
    ray_mask =np.zeros(sensor.sizes['nrays'], dtype=np.int)
    
    ray_mask_pixel = np.zeros(sensor.npixels.size, dtype=np.int)
    ray_mask_pixel[np.where(sensor.I.data > 1e-4)] = 1

    copied['weights'] = ('nrays',np.ones(sensor.nrays.size))#[sensor.pixel_index.data])
    copied['cloud_mask'] = ('nrays', ray_mask_pixel[sensor.pixel_index.data])
    sensor_list.append(copied)
    
def mean_ext_estimate(rte_grid, sensors, solar_mu, solar_azimuth,
                     chi=2/3, g=0.86, sun_distance_reflect=0.1,
                     sun_distance_transmit=0.3,
                     length_scale_method='max_height'):
    """
    Estimate the extinction of a cloud using diffusion theory.

    Given a masked volume `space_carved_volume`, the geometric distance
    between each point and the sun is calculated. The value of the geometric distance
    from the sun through the cloud at the first intersection of a sensor ray
    with the cloud volume is used to classify whether sensor pixels are
    observing shadowed or directly illuminated portions of the cloud.

    The mean of all 'shadowed' and 'illuminated' pixels is used to derive an
    optical diameter using diffusion theory and the extrapolation length `chi`
    and an asymmetry factor. This optical diameter is converted to an extinction
    using the length scale of the maximum chord length through the cloud in the solar
    direction. This length scale is chosen because it collapses to the relevant case
    for several geometries.
    """
    space_carver = pyshdom.space_carve.SpaceCarver(rte_grid)
    if isinstance(sensors, xr.Dataset):
        sensor_list = [sensors]
    elif isinstance(sensors, type([])):
        sensor_list = sensors
    elif isinstance(sensors, pyshdom.containers.SensorsDict):
        sensor_list = []
        for instrument in sensors:
            sensor_list.extend(sensors[instrument]['sensor_list'])

    volume = space_carver.carve(sensor_list, agreement=(0.0, 1.0), linear_mode=False)
    sundistance = space_carver.shadow_mask(volume.mask, sensor_list, solar_mu, solar_azimuth)

    reflected = []
    transmitted = []
    for sensor in sensor_list:
        reflected.extend(sensor.I.data[np.where((sensor.sun_distance.data < 0.1)&(sensor.I > 1e-2))])
        transmitted.extend(sensor.I.data[np.where((sensor.sun_distance.data >= 0.3)&(sensor.I > 1e-2))])
    
    if length_scale_method == 'sun_distance':
        radius = sundistance.sun_distance.data[np.where(sundistance.sun_distance > 0.0 )].max()
    elif length_scale_method == 'max_height':
        heights = rte_grid.z[np.where(volume.mask > 0.0)[-1]]
        radius = heights.max() - heights.min()

    tau_estimate = 2*chi*np.mean(reflected)/np.mean(transmitted)/(1.0-g)
    ext_estimate = tau_estimate/radius

    extinction = np.zeros(volume.mask.shape)
    extinction[np.where(volume.mask == 1.0)] = ext_estimate
    extinction = xr.Dataset(
        data_vars={
            'extinction': (['x', 'y', 'z'], extinction)
        },
        coords={
            'x': rte_grid.x,
            'y': rte_grid.y,
            'z': rte_grid.z,
        }
    )
    return extinction
    
extinction = mean_ext_estimate(rte_grid,sensor_list, -0.5, 0.0)

noise_std = np.std(solvers[0.86].medium['cloud'].extinction.data[np.where(solvers[0.86].medium['cloud'].extinction.data > 0.0)])
extinct_perturb = np.random.normal(loc=extinction.extinction.max(),
                                   scale=noise_std,
                                   size=extinction.extinction.shape)
ext_ref = extinction.extinction.data
ext_ref[0] = ext_ref[-1] = ext_ref[:,0] = ext_ref[:,-1] = 0.0
#ext_ref[np.where(ext_ref >0.0)] += extinct_perturb[np.where(ext_ref > 0.0)]
#ext_ref[np.where(ext_ref < 0.0)] = 0.0
extinction.extinction[:] = ext_ref

In [4]:
extinction.extinction.max()

<xarray.DataArray 'extinction' ()>
array(29.74613654)

In [5]:

# only take MSPI observations.
for name in sensors:
    if name != 'MSPI':
        sensors.popitem(name)
        
# make forward_sensors which will hold synthetic measurements from the evaluation of the forward model.
forward_sensors = sensors.make_forward_sensors()

# add an uncertainty model to the observations.
uncertainty = pyshdom.uncertainties.Uncertainty(np.diag(np.array([1e5,0.0,0.0,0.0])),'L2')
sensors.add_uncertainty_model('MSPI', uncertainty)

# prepare all of the static inputs to the solver just copy pasted from forward model
surfaces = OrderedDict()
numerical_parameters = OrderedDict()
sources = OrderedDict()
num_stokes = OrderedDict()
background_optical_scatterers = OrderedDict()
for key in forward_sensors.get_unique_solvers():
    surfaces[key] = solvers[key].surface
    numerical_parameters[key] = solvers[key].numerical_params
    sources[key] = solvers[key].source
    num_stokes[key] = solvers[key]._nstokes
    background_optical_scatterers[key] = {}


/Library/anaconda3/envs/pyshdom/lib/python3.9/site-packages/pyshdom/uncertainties.py:69: UserWarning: `inverse_covariance` of errors in uninvertible.It cannot be used to generate noise and will cause an exception to be raised if attempted.
  warnings.warn("`inverse_covariance` of errors in uninvertible."


In [6]:
# set the generator for the unknown scatterer using ground truth optical properties
# and unknown extinction.
# OpticalGenerator holds the fixed optical properties and forms a full set of optical properties
# when it is called with extinction as the argument.

# MicrophysicalGenerator does the same but taking microphysics as the input. However that
# needs an OpticalPropertyGenerator to map between microphysics and optical properties as well.
optprop = solvers[0.86].medium['cloud']

deriv_gen = pyshdom.medium.OpticalGenerator(rte_grid,'cloud', 0.86, 
                                            optprop.legcoef, optprop.ssalb,
                                           optprop.phase_weights, optprop.table_index,
                                           )

# UnknownScatterers is a container for all of the unknown variables.
unknown_scatterers = pyshdom.containers.UnknownScatterers()
unknown_scatterers.add_unknowns(['extinction'], deriv_gen)

In [7]:
# now we form state_gen which updates the solvers with an input_state.
# Note that `pyshdom.medium.StateGenerator` also takes as inputs a state_transform, a state_representation
# and a state_to_grid objects. These objects handle multi-variable
solvers_reconstruct = pyshdom.containers.SolversDict()
mask = np.zeros((rte_grid.x.size,rte_grid.y.size,rte_grid.z.size)).astype(np.bool)
mask[np.where(extinction.extinction.data > 0.0)] = 1.0
state_gen = pyshdom.medium.StateGenerator(solvers_reconstruct,
                                         unknown_scatterers, rte_grid,surfaces,
                                         numerical_parameters, sources, background_optical_scatterers,
                                         num_stokes, state_transform=None, state_to_grid=mask)

In [83]:
self = solvers[0.86]
#reshaped_ext = self._total_ext[:self._nbpts].reshape(self._nx1, self._ny1, self._nz)
reshaped_ext = self.medium['cloud'].extinction
cell_averaged_extinct = (reshaped_ext[1:, 1:, 1:] + reshaped_ext[1:, 1:, :-1] +   \
                         reshaped_ext[1:, :-1, 1:] + reshaped_ext[1:, :-1, :-1] + \
                         reshaped_ext[:-1, 1:, 1:] + reshaped_ext[:-1, 1:, :-1] + \
                         reshaped_ext[:-1, :-1, 1:] + reshaped_ext[:-1, :-1, :-1])/8.0

In [84]:
py.figure()
for i in range(0,cell_averaged_extinct.shape[-1],3):
    py.hist(cell_averaged_extinct[...,i].data[np.where(cell_averaged_extinct[...,i] > 0.0)],
           histtype='step',bins=np.linspace(0.0,130.0,20),label='{}'.format(i))
py.legend()
    

In [47]:
py.figure()
py.imshow(cell_averaged_extinct[12])

In [8]:
# get bounds automatically.
min_bounds, max_bounds = state_gen.transform_bounds()

In [9]:
# transform initial physical state to abstract state. 
# This is
x0 = extinction.extinction.data#np.zeros((rte_grid.x.size,rte_grid.y.size,rte_grid.z.size)) + 1e-1
a = state_gen._state_to_grid.inverse(x0, 'cloud', 'extinction')
x0 = state_gen.state_transform.inverse(a)

In [121]:
x0

array([28.55629108, 28.55629108, 28.55629108, ..., 28.55629108,
       28.55629108, 28.55629108])

In [139]:
state_gen.state_transform(x0)

array([29.74613654, 29.74613654, 29.74613654, ..., 29.74613654,
       29.74613654, 29.74613654])

In [140]:
state_gen(x0)

0.1 0.04692769985720488 False False


/Library/anaconda3/envs/pyshdom/lib/python3.9/site-packages/pyshdom/solver.py:1478: RuntimeWarning: No wavelength specified in optical properties. Make sure that the wavelength used isconsistent with 0.86
  warnings.warn("No wavelength specified in optical properties. "


In [10]:

objective_function = pyshdom.optimize.ObjectiveFunction.LevisApproxUncorrelatedL2(
    sensors, solvers_reconstruct, forward_sensors, unknown_scatterers, state_gen,
  state_gen.project_gradient_to_state,
    parallel_solve_kwargs={'n_jobs': 4, 'verbose': True},
  gradient_kwargs={'cost_function': 'L2', 'exact_single_scatter':True},
  uncertainty_kwargs={'add_noise': False},
  min_bounds=min_bounds, max_bounds=max_bounds)

In [20]:
rte_grid

<xarray.Dataset>
Dimensions:  (x: 32, y: 37, z: 30)
Coordinates:
  * x        (x) float64 0.0 0.02 0.04 0.06 0.08 0.1 ... 0.54 0.56 0.58 0.6 0.62
  * y        (y) float64 0.0 0.02 0.04 0.06 0.08 0.1 ... 0.64 0.66 0.68 0.7 0.72
  * z        (z) float64 0.0 0.44 0.48 0.52 0.56 0.6 ... 1.4 1.44 2.0 3.0 4.0
Data variables:
    delx     float64 0.02
    dely     float64 0.02

In [141]:
state_gen(x0)
forward_sensors.get_measurements(solvers_reconstruct)

/Library/anaconda3/envs/pyshdom/lib/python3.9/site-packages/pyshdom/solver.py:1478: RuntimeWarning: No wavelength specified in optical properties. Make sure that the wavelength used isconsistent with 0.86
  warnings.warn("No wavelength specified in optical properties. "


0.1 0.04692769985720488 False False


In [11]:
_, gradient = objective_function(x0)

/Library/anaconda3/envs/pyshdom/lib/python3.9/site-packages/pyshdom/solver.py:1478: RuntimeWarning: No wavelength specified in optical properties. Make sure that the wavelength used isconsistent with 0.86
  warnings.warn("No wavelength specified in optical properties. "


0.1 0.04692769985720488 False False
[1622446675.320618, 1622446676.154758, 1622446676.386637, 1622446676.462264, 1622446676.539957, 1622446687.1492522]


In [39]:
np.where(np.isnan(grad_shaped))

(array([5, 5, 5, 5]), array([36, 36, 36, 36]), array([ 6,  7,  9, 10]))

In [37]:
np.unravel_index(920,(extinction.extinction.shape))

(0, 30, 20)

In [40]:
solver = solvers_reconstruct[0.86]
grad_shaped=state_gen._state_to_grid(gradient,'cloud','extinction')
#grad_shaped = gradient.reshape(solver._grid.x.size, solver._grid.y.size, solver._grid.z.size)

py.figure()
py.imshow(grad_shaped[5])
py.colorbar()

In [14]:
%matplotlib qt
for im,im2 in zip(forward_sensors.get_images('MSPI'), sensors.get_images('MSPI')):
    py.figure()
    (im.I-im2.I).plot()

In [12]:
optimizer = pyshdom.optimize.Optimizer(objective_function, prior_fn=None)


In [32]:
x0

array([0., 0., 0., ..., 0., 0., 0.])

In [13]:
optimizer.minimize(x0)

0.1 0.04692769985720488 False False


/Library/anaconda3/envs/pyshdom/lib/python3.9/site-packages/pyshdom/solver.py:1478: RuntimeWarning: No wavelength specified in optical properties. Make sure that the wavelength used isconsistent with 0.86
  warnings.warn("No wavelength specified in optical properties. "


[1622446687.3207302, 1622446688.123911, 1622446688.3571699, 1622446688.439382, 1622446688.5324311, 1622446698.0156631]
0.1 0.04692728242309267 False False


/Library/anaconda3/envs/pyshdom/lib/python3.9/site-packages/pyshdom/solver.py:1478: RuntimeWarning: No wavelength specified in optical properties. Make sure that the wavelength used isconsistent with 0.86
  warnings.warn("No wavelength specified in optical properties. "


[1622446698.176322, 1622446698.9439669, 1622446699.169319, 1622446699.2396889, 1622446699.313718, 1622446708.743819]
0.1 0.046926864516718354 False False


...

[1622447008.6449091, 1622447009.5472052, 1622447009.7723782, 1622447009.8423848, 1622447009.9127831, 1622447019.251792]
0.1 0.015234008599980638 False False


/Library/anaconda3/envs/pyshdom/lib/python3.9/site-packages/pyshdom/solver.py:1478: RuntimeWarning: No wavelength specified in optical properties. Make sure that the wavelength used isconsistent with 0.86
  warnings.warn("No wavelength specified in optical properties. "


[1622447019.381515, 1622447020.2668211, 1622447020.485664, 1622447020.5569558, 1622447020.628035, 1622447030.224844]
0.1 0.015228094930737993 False False


/Library/anaconda3/envs/pyshdom/lib/python3.9/site-packages/pyshdom/solver.py:1478: RuntimeWarning: No wavelength specified in optical properties. Make sure that the wavelength used isconsistent with 0.86
  warnings.warn("No wavelength specified in optical properties. "


KeyboardInterrupt: 

In [21]:
state_gen._unknown_scatterers

UnknownScatterers([('cloud',
                    {'variable_name_list': ['extinction'],
                     'dataset_generator': <pyshdom.medium.OpticalGenerator at 0x134ac7d60>})])

In [23]:
solvers_reconstruct[0.86]._unknown_scatterer_indices

array([[2]], dtype=int32)

In [15]:
py.figure()
py.plot(solvers_reconstruct[0.86].medium['cloud'].extinction.data.ravel(), 
        solvers[0.86].medium['cloud'].extinction.data.ravel(),'x')

In [17]:
cond = np.where(solvers_reconstruct[0.86].medium['cloud'].extinction > 0.0)
one = solvers_reconstruct[0.86].medium['cloud'].extinction.data[cond]
ref = solvers[0.86].medium['cloud'].extinction.data[cond]
print(np.mean(one), np.mean(ref), np.mean(np.abs(one - ref))/np.mean(ref))

9.747940899135358 9.989887744836532 0.31471784498055466


In [24]:
py.figure()
solvers_reconstruct[0.86].medium['cloud'].extinction[12,:,:-3].plot()

In [25]:
py.figure()
solvers[0.86].medium['cloud'].extinction[12,:,:-3].plot()